<a href="https://colab.research.google.com/github/BruunoRangel/BruunoRangel/blob/main/Aula_8_Exerc%C3%ADcio_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Bruno Rangel Balbino dos santos "
print(f'Meu nome é {nome}')

Meu nome é Bruno Rangel Balbino dos santos 


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.7 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 68.5 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 20:28:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokens_indx = []
        self.y = []
        for text in texts:
            tokens_indx = tokenize(text, tokenizer)

            if len(tokens_indx) > context_size:
             for i in range(len(tokens_indx)-context_size):
                 self.tokens_indx.append(tokens_indx[i:i+context_size]) 
              
                 self.y.append(tokens_indx[i+context_size])
                
    def __len__(self):  
        return len(self.tokens_indx)

    def __getitem__(self, idx):
        
        return torch.tensor(self.tokens_indx[idx]).long(), torch.tensor(self.y[idx]).long()

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 20:28:32--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 142.251.12.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M  37.8MB/s    in 3.1s    

2022-05-25 20:28:37 (37.8 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [ ]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Truncating for debugging purposes.


In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super(LanguageModel, self).__init__()

        self.context_size = context_size
        self.embedding_dim = embedding_dim

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.embeddings_P = nn.Embedding(context_size, embedding_dim) 
        self.b_1 = nn.Parameter(torch.zeros(embedding_dim))
        self.a_1 = nn.Parameter(torch.randn(embedding_dim))
        self.eps = 1e-6

        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_0 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        hidden_size = embedding_dim
        self.linear1 = nn.Linear(embedding_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size, bias = False)
        self.relu = nn.ReLU()
       
     

    def forward(self, inputs, debug=False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0]
        position = torch.arange(0, self.context_size).unsqueeze(0).repeat(batch_size,1).to(inputs.device)
    
        embeds = self.embeddings(inputs) 
        embeds_position = self.embeddings_P(position) 

        X = embeds + embeds_position 
        mean = X.mean(-1, keepdim=True) 
        std = X.std(-1, keepdim=True)
        X = self.a_1 * (X - mean) / (std + self.eps) + self.b_1 #NormLayer

        X_1 = X[:,-1,:].unsqueeze(1) 
        
        Q = self.W_q(X_1) 
        d_k = Q.size(-1)

        K = self.W_k(X)     
  
        V = self.W_v(X)     
    
        scores = torch.matmul(Q,K.transpose(-2,-1))/math.sqrt(d_k) 
        probs = scores.softmax(dim=-1) 
        
        E = torch.matmul(probs, V) 
        
        E = self.W_0(E)
        logits = self.linear1(E.view(batch_size,-1)) 
        logits = self.relu(logits)
        logits = self.linear2(logits)

     

        return logits

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3834880


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30105
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 100_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29881.49, valid ppl: 29810.34
10000 steps; 10240000 examples so far; train ppl: 1193.65, valid ppl: 718.81
20000 steps; 20480000 examples so far; train ppl: 553.50, valid ppl: 456.72
30000 steps; 30720000 examples so far; train ppl: 407.45, valid ppl: 372.57
40000 steps; 40960000 examples so far; train ppl: 344.18, valid ppl: 324.82
50000 steps; 51200000 examples so far; train ppl: 308.80, valid ppl: 295.06
60000 steps; 61440000 examples so far; train ppl: 280.47, valid ppl: 274.24
70000 steps; 71680000 examples so far; train ppl: 262.12, valid ppl: 258.42
80000 steps; 81920000 examples so far; train ppl: 250.05, valid ppl: 245.54
90000 steps; 92160000 examples so far; train ppl: 234.44, valid ppl: 235.96


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')
#TEMPO DE TREINO 41min 

test perplexity: 219.02018805352748


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz a
Eu gosto de comer pizza pois me faz a sua
Eu gosto de comer pizza pois me faz a sua vida
Eu gosto de comer pizza pois me faz a sua vida.
Eu gosto de comer pizza pois me faz a sua vida. O
Eu gosto de comer pizza pois me faz a sua vida. O que
Eu gosto de comer pizza pois me faz a sua vida. O que é
Eu gosto de comer pizza pois me faz a sua vida. O que é o
Eu gosto de comer pizza pois me faz a sua vida. O que é o que
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é o
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é o que
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é o que é
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é o que é o
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é o que é o que
Eu gosto de comer pizza pois me faz a sua vida. O que é o que é o que é o que é
Eu gosto de comer pizza pois me faz a sua